In [ ]:
!pip install --upgrade  s3fs pandas  tqdm -q

In [ ]:
import pandas as pd
import tqdm
import boto3
import matplotlib

In [ ]:
! wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
! tar xvzf ag_news_csv.tgz

In [ ]:
train=pd.read_csv("ag_news_csv/train.csv", names=['category','title','text'])

In [ ]:
train.head()

In [ ]:
train = train.sample(axis='index',n=1000,random_state=100)

In [ ]:
labeldict={'1': 'MUNDO', '2' :  'ESPORTES', '3' : 'NEGÓCIOS', '4': 'CIÊNCIA'}
trainstr=train.astype(str)
trainstr['label']=trainstr['category'].replace(labeldict)

In [ ]:
dfout=trainstr[["label", 'text']]  

In [ ]:
dfout.head()

In [ ]:
dfout['label'].value_counts()

In [ ]:
translate = boto3.client("translate")
dfout['translated_text'] = dfout['text'].apply(lambda x: translate.translate_text(Text=x, SourceLanguageCode='en', TargetLanguageCode='pt').get('TranslatedText'))
dfout.head()

In [ ]:
dfout["text"] = dfout["translated_text"]
dfout.head()

In [ ]:
dfout=dfout[["label", "text"]]
dfout.head()

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "datasets"
file_name = f"s3://{bucket}/{prefix}/custom_classification.csv"
print(file_name)

In [ ]:
dfout.to_csv(file_name, header=False, index=False )